### Import Dependencies

In [21]:
import sys
sys.path.append("../")
import pandas as pd

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model,should_use_reasoning_model
from IPython.display import display, Markdown

### Loading data from sample.txt file

In [22]:
file_path = '../data/raw/Sample Messages.txt'

with open(file_path, 'r', encoding='utf-8') as f:
    messages = [line.strip() for line in f if line.strip()]

print(f"Loaded {len(messages)} messages\n")

Loaded 50 messages



### Adding few-shot examples

In [23]:
few_shot_examples = """
Message: BREAKING: Kelani River level has reached 9 meters.
Output: District: Colombo | Intent: Info | Priority: Low

Message: SOS: 5 people trapped on a roof in Ja-Ela. Water rising fast.
Output: District: Gampaha | Intent: Rescue | Priority: High

Message: Gampaha hospital is requesting drinking water for patients.
Output: District: Gampaha | Intent: Supply | Priority: Low

Message: Update: Road cleared near Peradeniya. Traffic moving slowly.
Output: District: Kandy | Intent: Info | Priority: Low

Message: We are trapped in the attic with 3 kids. Water entering.
Output: District: None | Intent: Rescue | Priority: High

Message: Donation drive happening at the town hall tomorrow.
Output: District: None | Intent: Other | Priority: Low

Message: URGENT: Landslide in Badulla. 2 houses buried.
Output: District: Badulla | Intent: Rescue | Priority: High

Message: The Navy has deployed boats to the Kelaniya area.
Output: District: Colombo | Intent: Info | Priority: Low

Message: The army is distributing food in the affected areas.
Output: District: None | Intent: Info | Priority: Low
"""


#### Refactor the classification prompt using few_shot.v1.

In [24]:
prompt_text, spec = render(
    "few_shot.v1",
    role="You are a disaster response message classifier for Sri Lankan flood and landslide emergencies.",
    examples=few_shot_examples,
    query="{message}",
    constraints = """
        - Output EXACTLY one line in this format: District: [Name] | Intent: [Category] | Priority: [High/Low]
        - ALWAYS include the prefix 'District:' even if the value is None
        - Do NOT repeat the message, do NOT add "Message:" or "Output:"
        - Do NOT include extra lines, explanations, or any extra words
        - Use ONLY these Intent categories: Rescue, Supply, Info, Other
            - Rescue = immediate danger to human life (trapped, injured, SOS, stranded)
            - Supply = explicit requests or needs for food, water, medicine, clothing, or shelter
            - Info = news reports, warnings, status updates, or aid already distributed
            - Other = prayers, opinions, missing pets, unrelated messages
        - District MUST be a valid Sri Lankan district
            - If a city/town is mentioned, infer its district (e.g., Ja-Ela → Gampaha)
            - If NO location is present, output District: None
        - Priority = High ONLY for Rescue messages
        - Priority = Low for all other messages
        - Supply intent is ONLY for explicit requests; reports of aid already being distributed are Info, NOT Supply
        - DO NOT confuse news reports with SOS calls
        - If any rule is violated, the output is invalid
        """,
    format="District: [Name] | Intent: [Category] | Priority: [High/Low]"
)


### Iterate through the loaded messages, classify each one using your few_shot.v1, and store the results. 

In [25]:
model = pick_model('groq', 'general')
client = LLMClient('groq', model)

classifications=[]

for msg in messages:
    # Replace the {message} placeholder with actual message
    full_prompt = prompt_text.replace("{message}", msg)
    
    msg_payload = [{'role': 'user', 'content': full_prompt}]
    
    # Get classification
    response = client.chat(msg_payload, temperature=0.0)
    classification = response['text'].strip()
    print(classification)
    classifications.append(classification)
    

District: Colombo | Intent: Info | Priority: Low
District: Gampaha | Intent: Rescue | Priority: High
District: Kandy | Intent: Info | Priority: Low
District: Gampaha | Intent: Supply | Priority: Low
District: Colombo | Intent: Info | Priority: Low
District: None | Intent: Rescue | Priority: High
District: Gampaha | Intent: Info | Priority: Low
District: None | Intent: Rescue | Priority: High
District: None | Intent: Other | Priority: Low
District: None | Intent: Info | Priority: Low
District: Badulla | Intent: Rescue | Priority: High
District: None | Intent: Other | Priority: Low
District: Colombo | Intent: Info | Priority: Low
District: Colombo | Intent: Rescue | Priority: High
District: None | Intent: Info | Priority: Low
District: Gampaha | Intent: Supply | Priority: Low
District: Ratnapura | Intent: Rescue | Priority: High
District: None | Intent: Info | Priority: Low
District: None | Intent: Other | Priority: Low
District: None | Intent: Other | Priority: Low
District: None | Inte

### Save the above classified results into a CSV/Excel file using pandas named output/classified_messages.xlsx 

In [27]:
data=[]
for d in classifications:
    d = d.split('|')
    result={
        'District':d[0].split(':')[1].strip(),
        'Intent':d[1].split(':')[1].strip(),
        'Priority':d[2].split(':')[1].strip()
    }
    data.append(result)

df = pd.DataFrame(data)
df.to_excel('../data/output/classified_messages.xlsx',index=False)